# Day 2B: Advanced Agent Tool Patterns

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Understand how to connect to external MCP (Model Context Protocol) servers
- Implement long-running operations that can pause agent execution for human input
- Build resumable workflows that maintain state across conversation breaks
- Master when and how to use these advanced patterns in production

## 📚 Prerequisites

- Completion of **Day 1A: Introduction to Agent Architectures**
- Completion of **Day 1B: Advanced Agent Architectures** (recommended)
- Completion of **Day 2A: Agent Tools** (recommended)
- Google API key (stored in `.env` file)
- Node.js installed (for MCP servers)

---


## Introduction: Beyond Basic Tools

In Day 2A, you learned about function tools and agent tools. These are great for many use cases, but what about:

- **Connecting to external systems** like databases, APIs, or services?
- **Operations that take hours or days** to complete?
- **Workflows that need to pause** for human approval or compliance?
- **Applications that must remember state** across conversation breaks?

This notebook covers three advanced patterns that solve these real-world challenges:

1. **Model Context Protocol (MCP)** - For connecting to external systems
2. **Long-Running Operations** - For workflows that need human input or take time
3. **Resumable Workflows** - For applications that must maintain state

---


## Setting Up Our Environment

Let's install the required packages:

In [122]:
import sys
import subprocess
import importlib

def install_package(package):
    """Install a Python package using pip"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check if python-dotenv is installed, install if not
try:
    import dotenv
    print("✅ python-dotenv is already installed")
except ImportError:
    print("Installing python-dotenv...")
    install_package("python-dotenv")
    import dotenv
    print("✅ python-dotenv installed successfully")

# Check if google-adk is installed, install if not
try:
    import google.adk.agents
    print("✅ google-adk is already installed")
except ImportError:
    print("Installing google-adk...")
    install_package("google-adk")
    import google.adk.agents
    print("✅ google-adk installed successfully")

# Check if google-genai is installed, install if not
try:
    import google.genai
    print("✅ google-genai is already installed")
except ImportError:
    print("Installing google-genai...")
    install_package("google-genai")
    import google.genai
    print("✅ google-genai installed successfully")

# Check if mcp is installed, install if not
try:
    import mcp
    print("✅ mcp is already installed")
except ImportError:
    print("Installing mcp...")
    install_package("mcp")
    import mcp
    print("✅ mcp installed successfully")

print("\n🎉 All required packages are installed and ready!")

✅ python-dotenv is already installed
✅ google-adk is already installed
✅ google-genai is already installed
✅ mcp is already installed

🎉 All required packages are installed and ready!


## Setting Up Authentication

Load the API key from your `.env` file:

In [123]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

try:
    # Get API key from environment variable (from .env file)
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in environment variables")
    
    # Set the API key for ADK
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    print("✅ Gemini API key setup complete.")
    print("🔑 Your API key is loaded and ready to use.")
    
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have set 'GOOGLE_API_KEY' in your .env file. Details: {e}")
    print("\n💡 To fix this:")
    print("1. Create a .env file in the same directory as this notebook")
    print("2. Add this line to your .env file: GOOGLE_API_KEY=your_actual_api_key_here")
    print("3. Replace 'your_actual_api_key_here' with your real Google API key")

✅ Gemini API key setup complete.
🔑 Your API key is loaded and ready to use.


## Importing ADK Components

For working with tools, we need additional components:

In [124]:
import uuid
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters
from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")
print("📦 Available components:")
print("  • LlmAgent - Advanced agent class with tool support")
print("  • McpToolset - For MCP server integration")
print("  • ToolContext - Access tool execution context")
print("  • App - For resumable workflows")
print("  • FunctionTool - For custom function tools")

✅ ADK components imported successfully.
📦 Available components:
  • LlmAgent - Advanced agent class with tool support
  • McpToolset - For MCP server integration
  • ToolContext - Access tool execution context
  • App - For resumable workflows
  • FunctionTool - For custom function tools


## Configuring Retry Options

When working with external services and LLMs, you may encounter transient errors. Let's configure retry options to make our agents more reliable:

In [125]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

print("✅ Retry configuration created.")
print(f"🔄 Will retry up to {retry_config.attempts} times for errors: {retry_config.http_status_codes}")

✅ Retry configuration created.
🔄 Will retry up to 5 times for errors: [429, 500, 503, 504]


---

## 🧰 Section 2: Model Context Protocol (MCP)

So far, you have learned how to create custom functions for your agents. But connecting to external systems (GitHub, databases, Slack) requires writing and maintaining API clients.

**Model Context Protocol (MCP)** is an open standard that lets agents use community-built integrations. Instead of writing your own integrations and API clients, just connect to an existing MCP server.

MCP enables agents to:

✅ **Access live, external data** from databases, APIs, and services without custom integration code  
✅ **Leverage community-built tools** with standardized interfaces  
✅ **Scale capabilities** by connecting to multiple specialized servers
### 2.1: How MCP Works

MCP connects your agent (the **client**) to external **MCP servers** that provide tools:

- **MCP Server**: Provides specific tools (like image generation, database access)
- **MCP Client**: Your agent that uses those tools
- **All servers work with the same way** - standardized interface

### Architecture

```
     ┌──────────────────┐
     │   Your Agent     │
     │   (MCP Client)   │
     └────────┬─────────┘
              │
              │ Standard MCP Protocol
              │
         ┌────┴────┬───────┬────────┐
         │         │       │        │
         ▼         ▼       ▼        ▼
     ┌────────┐ ┌─────┐ ┌──────┐ ┌─────┐
     │ GitHub │ │Slack│ │ Maps │ │ ... │
     │ Server │ │ MCP │ │ MCP  │ │     │
     └────────┘ └─────┘ └──────┘ └─────┘
```
### 2.2: Using MCP with Your Agent

The workflow is simple:

1. Choose an MCP Server and tool
2. Create an MCP Toolset (configure connection)
3. Add it to your agent
4. Run and test the agent

#### Step 1: Choose MCP Server

For this demo, we'll use the **[Everything MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/everything)** - an npm package (`@modelcontextprotocol/server-everything`) designed for testing MCP integrations.

It provides a `getTinyImage` tool that returns a simple test image (16x16 pixels, Base64-encoded). **Find more servers:** [modelcontextprotocol.io/examples](https://modelcontextprotocol.io/examples)

‼️ **NOTE**: This is a demo server to learn MCP. In production, you'll use servers for Google Maps, Slack, Discord, etc.

In [126]:
# MCP integration with Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")
print("📌 Connected to Everything MCP Server")
print("🔧 Available tool: getTinyImage")

✅ MCP Tool created
📌 Connected to Everything MCP Server
🔧 Available tool: getTinyImage


#### Behind the Scenes:

1. **Server Launch**: ADK runs `npx -y @modelcontextprotocol/server-everything`
2. **Handshake**: Establishes stdio communication channel
3. **Tool Discovery**: Server tells ADK: "I provide getTinyImage" functionality
4. **Integration**: Tools appear in agent's tool list automatically
5. **Execution**: When agent calls `getTinyImage()`, ADK forwards to MCP server
6. **Response**: Server result is returned to agent seamlessly

**Why This Matters**: You get instant access to tools without writing integration code!

#### Step 2: Add MCP tool to agent

Let's add the `mcp_server` to the agent's tool array and update the agent's instructions to handle requests to generate tiny images.

In [127]:
# Create image agent with MCP integration
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="""You are an image generation assistant. When users ask for images:
    1. Use the getTinyImage tool to generate a 16x16 pixel image
    2. Describe what the image shows in simple terms
    3. If they ask for modifications, explain that you can only generate new images
    
    Note: The tool returns base64-encoded image data.""",
    tools=[mcp_image_server],
)

print("✅ Image agent created with MCP tool")
print("🎨 Agent ready to generate images via MCP server")

✅ Image agent created with MCP tool
🎨 Agent ready to generate images via MCP server


#### Step 3: Test the agent

Let's test our image generation agent:

In [128]:
# Create a runner for the agent
from google.adk.runners import InMemoryRunner
runner = InMemoryRunner(agent=image_agent)

print("🚀 Testing image generation agent...")
print("📝 Request: Generate a tiny test image")

# Test the agent
response = await runner.run_debug("Generate a tiny test image", verbose=True)

print("\n🤖 Agent response received")
print("📊 Processing response...")

🚀 Testing image generation agent...
📝 Request: Generate a tiny test image

 ### Created new session: debug_session_id

User > Generate a tiny test image


/Users/tdiaconescu/.direnv/python-3.11/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > This is a tiny image.

🤖 Agent response received
📊 Processing response...


#### Display the generated image

The server returns base64-encoded image data. Let's decode and display it:

In [129]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

### 2.3: Extending to Other MCP Servers

The same pattern works for any MCP server - only the `connection_params` change. Here are some examples:

#### 👉 Kaggle MCP Server - For dataset and notebook operations

Kaggle provides an MCP server that lets your agents interact with Kaggle datasets, notebooks, and competitions.

**Connection example:**
```python
McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                '-y',
                'mcp-remote',
                'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=30,
    )
)
```

**What it provides:**
- 📊 Search and download Kaggle datasets
- 📓 Access notebook metadata
- 🏆 Query competition information etc.

**Learn more:** [Kaggle MCP Documentation](https://www.kaggle.com/docs/mcp)

#### 👉 GitHub MCP Server - For PR/Issue analysis

```python
McpToolset(
    connection_params=StreamableHTTPServerParams(
        url="https://api.githubcopilot.com/mcp/",
        headers={
            "Authorization": f"Bearer {GITHUB_TOKEN}",
            "X-MCP-Toolsets": "all",
            "X-MCP-Readonly": "true"
        },
    ),
)
```

**More resources:** [ADK Third-party Tools Documentation](https://google.github.io/adk-docs/tools/third-party/)

---

## 🔄 Section 3: Long-Running Operations (Human-in-the-Loop)

So far, all tools execute and return immediately:

```
User asks → Agent calls tool → Tool returns result → Agent responds
```

**But what if your tools are long-running or you need human approval before completing an action?**

Example: A shipping agent should ask for approval before placing a large order.

```
User asks → Agent calls tool → Tool PAUSES and asks human → Human approves → Tool completes → Agent responds
```

This is called a **Long-Running Operation (LRO)** - a tool that needs to pause, wait for external input (human approval), then resume.

### When to Use Long-Running Operations

| Use Case | Example | Why LRO Needed |
|----------|-------------|----------------|
| 💰 **Financial transactions** | Large bank transfers | Regulatory compliance |
| 🗑️ **Bulk operations** | Delete 1000 records | Prevent accidental data loss |
| 📋 **Compliance checkpoints** | Medical diagnoses | Human verification required |
| 💸 **High-cost actions** | Spin up 50 servers | Budget approval |
| ⚠️ **Irreversible operations** | Permanently delete account | Final confirmation |

### 3.1: What We're Building Today

Let's build a **shipping coordinator agent** with one tool that:

- Auto-approves small orders (≤5 containers)
- **Pauses and asks for approval** on large orders (>5 containers)
- Completes or cancels based on approval decision

This demonstrates the core long-running operation pattern: **pause → wait for human input → resume**.

### 3.2: The Shipping Tool with Approval Logic

Here's the complete function:

In [130]:
# Define the shipping order tool with approval logic
LARGE_ORDER_THRESHOLD = 5

def place_shipping_order(num_containers: int, destination: str, tool_context: ToolContext) -> dict:
    """Place a shipping order with approval logic.
    
    Args:
        num_containers: Number of containers to ship
        destination: Destination port/city
        tool_context: ADK tool context for requesting confirmation
    
    Returns:
        dict: Order status and details
    """
    
    # Check if this is the first call or a resumed call
    if not tool_context.tool_confirmation:
        # First call - check if approval is needed
        if num_containers > LARGE_ORDER_THRESHOLD:
            # Large order - request human approval
            tool_context.request_confirmation(
                hint=f"Large order detected: {num_containers} containers to {destination}. Requires approval.",
                payload={
                    "title": "Shipping Order Approval",
                    "description": f"Approve shipping {num_containers} containers to {destination}"
                },
            )
            return {
                "status": "pending",
                "message": f"Awaiting approval for {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-PENDING",
            }
        else:
            # Small order - auto-approve
            return {
                "status": "approved",
                "message": f"Auto-approved small order: {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-AUTO",
            }
    else:
        # Resumed call - check the approval decision
        if tool_context.tool_confirmation.confirmed:
            # Human approved - complete the order
            return {
                "status": "approved",
                "message": f"Human approved: {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-HUMAN",
            }
        else:
            # Human rejected - cancel the order
            return {
                "status": "rejected",
                "message": f"Human rejected: {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-CANCELLED",
            }

print("✅ Shipping order tool defined with approval logic")
print(f"📋 Large order threshold: {LARGE_ORDER_THRESHOLD} containers")

✅ Shipping order tool defined with approval logic
📋 Large order threshold: 5 containers


#### The `ToolContext` Parameter

Notice the function signature includes `tool_context: ToolContext`. ADK automatically provides this object when your tool runs. It gives you two key capabilities:

1. **Request approval**: Call `tool_context.request_confirmation()`
2. **Check approval status**: Read `tool_context.tool_confirmation`

```python
LARGE_ORDER_THRESHOLD = 5
```

### 3.3: Understanding the Code

Now that you've seen the complete function, let's break down how it works:

##### How to Three Scenarios Work

The tool handles three scenarios by checking `tool_context.tool_confirmation`:

**Scenario 1: Small order (≤5 containers)**: Returns immediately with auto-approved status
- `tool_context.tool_confirmation` is never checked

**Scenario 2: Large order - FIRST CALL**
- Tool detects it's a first call: `if not tool_context.tool_confirmation:`
- Calls `request_confirmation()` to request human approval
- Returns `{'status': 'pending', ...}` immediately
- **ADK automatically creates `adk_request_confirmation` event**
- Agent execution pauses - waiting for human decision

**Scenario 3: Large order - RESUMED CALL**
- Tool detects it's resuming: `if not tool_context.tool_confirmation:` is now False
- Checks human decision: `tool_context.tool_confirmation.confirmed`
- If True → Returns approved status
- If False → Returns rejected status

**Key insight**: Between the two calls, your workflow code (in Section 4) must detect the `adk_request_confirmation` event and resume with the approval decision.

### 3.4: Create the Agent

Let's add the tool to an Agent. The tool decides internally when to request approval based on the order size.

In [131]:
# Create the shipping agent with pausable tool
shipping_agent = LlmAgent(
    name="shipping_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a shipping coordinator assistant.

   When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
""",
    tools=[FunctionTool(func=place_shipping_order)],
)

print("✅ Shipping Agent created!")
print("🚢 Agent ready to handle shipping orders with approval logic")

✅ Shipping Agent created!
🚢 Agent ready to handle shipping orders with approval logic


### 3.5: Wrap in a Resumable App

**The problem**: A regular `LlmAgent` is stateless - each call is independent with no memory of previous interactions. If a tool requests approval, the agent can't remember what it was doing.

**The solution**: Wrap your agent in an **`App`** with **resumability enabled**. The App adds a persistence layer that saves and restores state.

#### What Gets Saved When a Tool Pauses

- All conversation messages so far
- Which tool was called (`place_shipping_order`)
- Tool parameters (10 containers, Rotterdam)
- Where exactly it paused (waiting for approval)

When you resume, the App loads this saved state so the agent continues exactly where it left off - as if no time passed.

In [132]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
from google.adk.apps.app import App, ResumabilityConfig

shipping_app = App(
    name="shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")
print("🔄 State persistence enabled for shipping workflow")

✅ Resumable app created!
🔄 State persistence enabled for shipping workflow


/var/folders/yc/qb64jwtj4p7b9plvr1pv_3q80000gn/T/ipykernel_1135/1812355229.py:7: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


### 3.6: Create Session and Runner with the App

Pass `app=shipping_app` instead of `agent=...` so that the runner knows about resumability.

In [133]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# Create a session service for the app
session_service = InMemorySessionService()

# Create runner with the resumable app
shipping_runner = Runner(
    app=shipping_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")
print("🎯 Ready to test long-running operations with state persistence")

✅ Runner created!
🎯 Ready to test long-running operations with state persistence


#### ✅ Recap: Your Pausable Shipping Agent is Complete!

You've created:

1. ✅ A tool that can pause for approval (`place_shipping_order`)
2. ✅ An agent that uses this tool (`shipping_agent`)
3. ✅ A resumable app that saves state (`shipping_app`)
4. ✅ A runner that can handle pause/resume (`shipping_runner`)

**Next step**: Build the workflow code and test that our Agent detects pauses and handles approvals.

### 4.1: ⚠️ The Critical Part - Handling Events in Your Workflow

The agent won't automatically handle pause/resume. **Every long-running operation workflow requires you to:**

1. **Detect the pause**: Check if events contain `adk_request_confirmation`
2. **Get the human decision**: In production, show UI and wait for user click. Here, we simulate it.
3. **Resume the agent**: Send the decision back with the saved `invocation_id`

### 4.2: Understand Key Technical Concepts

Let's review the key concepts we've learned:

#### 👉 `events`** - ADK creates events as the agent executes. Tool calls, model responses, function results - all become events

#### 👉 `adk_request_confirmation` event** - This event is special - it signals "pause here!"
- Automatically created by ADK when your tool calls `request_confirmation()`
- Contains `invocation_id`
- Your workflow must detect this event to know that the agent paused

#### 👉 `invocation_id`** - Every call to `run_async()` gets a unique `invocation_id` (like "abc123")
- When a tool pauses, you save this ID
- When resuming, pass the same ID so ADK knows which execution to continue
- Without it, ADK would start a NEW execution instead of resuming the paused one

### 4.3: Helper Functions to Process Events

These handle the event iteration logic for you:

In [134]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

def print_agent_response(events):
    """Print the agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

def create_approval_response(approval_info, approved):
    """Create an approval response message."""
    from google.genai import types
    
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )

print("✅ Helper functions defined!")
print("🔧 Ready to handle approval workflow events")

✅ Helper functions defined!
🔧 Ready to handle approval workflow events


### 4.4: The Workflow Function - Let's Tie It All Together!

The `run_shipping_workflow()` function orchestrates the entire approval flow.

In [135]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
    """Runs a shipping workflow with approval handling.

    Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
    """
    
    print(f"\n{'='*60}")
    print(f"User > {query}")
    
    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"
    
    # Create session
    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )
    
    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []
    
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns a special `adk_request_confirmation` event
    # -----------------------------------------------------------------------------------------------
    async for event in shipping_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)
    
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    # -----------------------------------------------------------------------------------------------
    approval_info = check_for_approval(events)
    
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    # -----------------------------------------------------------------------------------------------
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")
        
        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")
    
    # -----------------------------------------------------------------------------------------------
    # STEP 4: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
    # -----------------------------------------------------------------------------------------------
    else:
        print_agent_response(events)
    
    print(f"{'='*60}\n")

print("✅ Workflow function ready!")
print("🎯 Ready to test complete approval workflow")

✅ Workflow function ready!
🎯 Ready to test complete approval workflow


### 4.5: Demo: Testing the Workflow

Now, let's run our demos. Notice how much cleaner and easier to read they are. All of the complex logic for pausing and resuming is now hidden away in our `run_workflow` helper function, allowing us to focus on the tasks we want the agent to perform.

In [136]:
# Demo 1: It's a small order. The Agent receives auto-approved status from the tool
await run_shipping_workflow("Ship 3 containers to Singapore")

# Demo 2: Workflow simulates human decision: APPROVE ✅
await run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=True)

# Demo 3: Workflow simulates human decision: REJECT ❌
await run_shipping_workflow("Ship 8 containers to Los Angeles", auto_approve=False)

print("\n🎉 All demos completed!")
print("💡 Notice how the workflow handles each scenario automatically")


User > Ship 3 containers to Singapore


Agent > 3 containers to Singapore have been approved and shipped. The order ID is ORD-3-AUTO.


User > Ship 10 containers to Rotterdam


/Users/tdiaconescu/.direnv/python-3.11/lib/python3.11/site-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/Users/tdiaconescu/.direnv/python-3.11/lib/python3.11/site-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅

Agent > Shipping order approved. Order ID: ORD-10-HUMAN. 10 containers will be shipped to Rotterdam.


User > Ship 8 containers to Los Angeles


⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > I'm sorry, but your shipping order for 8 containers to Los Angeles has been rejected. The order ID is ORD-8-CANCELLED.


🎉 All demos completed!
💡 Notice how the workflow handles each scenario automatically


#### 📊 Execution Timeline

Here's an example trace of the whole workflow:

```
TIME 1: User sends "Ship 10 containers to Rotterdam"
        ↓
TIME 2: Workflow calls shipping_runner.run_async(...)
        ADK assigns a unique invocation_id = "abc123"
        ↓
TIME 3: The Agent receives the user message, decides to use the place_shipping_order tool
        ↓
TIME 4: ADK calls place_shipping_order(10, "Rotterdam", tool_context)
        ↓
TIME 5: The Tool checks: num_containers (10) > 5
        The Tool calls tool_context.request_confirmation(...)
        ↓
TIME 6: The Tool returns {'status': 'pending', ...}
        ↓
TIME 7: ADK creates the adk_request_confirmation event with invocation_id="abc123"
        ↓
TIME 8: Workflow detects the event via check_for_approval()
        Saves the approval_id and invocation_id="abc123"
        ↓
TIME 9: Workflow gets the human decision → True (approve)
        ↓
TIME 10: Workflow calls shipping_runner.run_async(..., invocation_id="abc123")
        Passes the approval decision as a FunctionResponse
        ↓
TIME 11: ADK sees the invocation_id="abc123" - knows to RESUME (instead of starting a new one)
        Loads the saved state from TIME 7
        ↓
TIME 12: ADK calls place_shipping_order again with the same parameters
        But now tool_context.tool_confirmation.confirmed = True
        ↓
TIME 13: The Tool returns {'status': 'approved', 'order_id': 'ORD-10-HUMAN', ...}
        ↓
TIME 14: The Agent receives the result and responds to the user
```

**Key point**: The `invocation_id` is how ADK knows to resume a paused execution instead of starting a new one.

---

## 🔄 Section 4: Resumable Workflows

Long-running operations solve the "pause and resume" problem. But what if:

- **User closes browser** during a paused workflow?
- **Server restarts** while waiting for approval?
- **Multi-day processes** that span multiple sessions?

**Resumable workflows** solve this by persisting state so agents can continue exactly where they left off - even hours or days later.


### When to Use Resumable Workflows

| Use Case | Example | Why Resumable Needed |
|----------|---------|---------------------|
| 📅 **Multi-day approvals** | Legal document review | Review happens over days |
| 🔄 **Interruptible processes** | CI/CD pipeline deployment | Can be paused/resumed |
| 📱 **Mobile workflows** | Field service requests | User on the go |
| 🌐 **Web applications** | E-commerce checkout | Save cart for later |

### 4.1: How Resumable Workflows Work

The key is the **`App`** class with **`ResumabilityConfig`**:

1. **State Persistence**: Automatically saves conversation history, tool calls, and execution state
2. **Session Management**: Each workflow gets a unique session ID for tracking
3. **Recovery**: Load and continue from exact pause point

### Architecture

```
        ┌───────────────────────────────────┐
        │        Your App (Resumable)       │
        │  ┌─────────────────────────────┐  │
        │  │        State Persistence    │  │
        │  │  ┌─────────┬──────────┐     │  │
        │  │  │ Session │ Tool     │     │  │
        │  │  │ Service │ State    │     │  │
        │  │  └─────────┴──────────┘     │  │
        │  └─────────────────────────────┘  │
        │               │                   │
        │        ┌──────┴───────────┐       │
        │        │    Your Agent    │       │
        │        └──────────────────┘       │
        └───────────────────────────────────┘
```

**Key insight**: The `App` wrapper is what enables state persistence. Without it, agents are stateless and can't resume.

### 4.2: Building a Multi-Step Approval Workflow

Let's extend our shipping example to demonstrate resumable workflows across multiple steps and sessions.

In [137]:
# Enhanced shipping tool with multi-step workflow
def place_shipping_order_with_tracking(num_containers: int, destination: str, tool_context: ToolContext) -> dict:
    """Enhanced shipping order with tracking and multi-step approval.
    
    Args:
        num_containers: Number of containers to ship
        destination: Destination port/city
        tool_context: ADK tool context for requesting confirmation
    
    Returns:
        dict: Order status and details
    """
    
    # Check if this is the first call or a resumed call
    if not tool_context.tool_confirmation:
        # First call - check if approval is needed
        if num_containers > LARGE_ORDER_THRESHOLD:
            # Large order - request human approval
            tool_context.request_confirmation(
                hint=f"Large order detected: {num_containers} containers to {destination}. Requires approval.",
                payload={
                    "title": "Shipping Order Approval",
                    "description": f"Approve shipping {num_containers} containers to {destination}"
                },
            )
            return {
                "status": "pending_approval",
                "message": f"Awaiting approval for {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-PENDING",
                "step": "approval_required",
            }
        else:
            # Small order - auto-approve
            return {
                "status": "approved",
                "message": f"Auto-approved small order: {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-AUTO",
                "step": "completed",
            }
    else:
        # Resumed call - check the approval decision
        if tool_context.tool_confirmation.confirmed:
            # Human approved - proceed to next step
            return {
                "status": "approved",
                "message": f"Human approved: {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-HUMAN",
                "step": "scheduling",
            }
        else:
            # Human rejected - cancel the order
            return {
                "status": "rejected",
                "message": f"Human rejected: {num_containers} containers to {destination}",
                "order_id": f"ORD-{num_containers}-CANCELLED",
                "step": "cancelled",
            }

print("✅ Enhanced shipping order tool defined with multi-step workflow")

✅ Enhanced shipping order tool defined with multi-step workflow


### 4.3: Creating a Resumable Multi-Step Agent

Let's create an agent that can handle complex, multi-step workflows:

In [138]:
# Create multi-step shipping agent
multi_step_agent = LlmAgent(
    name="multi_step_shipping_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a multi-step shipping coordinator assistant.

   Workflow:
   1. Check order size and request approval if needed
   2. If approved, schedule the shipment
   3. Confirm scheduling and provide tracking details
   4. Handle each step clearly with status updates

   Always:
   - Use the place_shipping_order_with_tracking tool
   - Track the current step in your responses
   - Provide clear next steps or completion status
""",
    tools=[FunctionTool(func=place_shipping_order_with_tracking)],
)

# Create resumable app for multi-step workflow
multi_step_app = App(
    name="multi_step_shipping_coordinator",
    root_agent=multi_step_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Multi-step agent and resumable app created!")
print("🔄 Ready for complex, resumable workflows")

✅ Multi-step agent and resumable app created!
🔄 Ready for complex, resumable workflows


/var/folders/yc/qb64jwtj4p7b9plvr1pv_3q80000gn/T/ipykernel_1135/3669139390.py:25: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


### 4.4: Testing Resumable Workflows

Let's test our resumable workflow with a complex scenario that spans multiple steps:

In [139]:
# Test resumable workflow with a large order
print("🧪 Testing resumable workflow...")
print("📋 Scenario: Large order requiring approval and scheduling")

# This would normally be run in separate sessions, but we'll simulate it here
session_id = f"resumable_{uuid.uuid4().hex[:8]}"
await session_service.create_session(
    app_name="shipping_coordinator", 
    user_id="test_user", 
    session_id=session_id
)

# Step 1: Initial request (will pause for approval)
query1 = "Ship 15 containers to Hamburg"
events1 = []
async for event in shipping_runner.run_async(
    user_id="test_user",
    session_id=session_id,
    new_message=types.Content(role="user", parts=[types.Part(text=query1)])
):
    events1.append(event)

# Check for approval request
approval_info = check_for_approval(events1)
if approval_info:
    print(f"⏸️  Workflow paused at step: approval_required")
    
    # Step 2: Resume with approval (simulate user coming back later)
    print("\n🔄 Simulating user returning after 1 hour to approve...")
    events2 = []
    async for event in shipping_runner.run_async(
        user_id="test_user",
        session_id=session_id,
        new_message=create_approval_response(approval_info, True),
        invocation_id=approval_info["invocation_id"],
    ):
        events2.append(event)
    
    print("\n✅ Workflow resumed and completed!")
    print_agent_response(events2)
else:
    print_agent_response(events1)

print("\n🎉 Resumable workflow test completed!")
print("💡 Key insight: The agent remembered its state across the pause/resume cycle")

🧪 Testing resumable workflow...
📋 Scenario: Large order requiring approval and scheduling


⏸️  Workflow paused at step: approval_required

🔄 Simulating user returning after 1 hour to approve...

✅ Workflow resumed and completed!
Agent > I have approved your request to ship 15 containers to Hamburg. The order ID is ORD-15-HUMAN.

🎉 Resumable workflow test completed!
💡 Key insight: The agent remembered its state across the pause/resume cycle


---

## 🎯 Exercise: Build a Production-Ready Agent

Now it's your turn! Combine all three patterns to build a production-ready agent:

**Task:** Create a document processing agent that:
1. Uses MCP to connect to a document storage service
2. Requires approval for processing sensitive documents
3. Can be resumed if the workflow is interrupted

**Requirements:**
- Use the `place_shipping_order_with_tracking` pattern for approval
- Create an MCP toolset for a hypothetical document service
- Wrap everything in a resumable app
- Test with both small and large document requests

**Hint:** Start with the multi-step agent template and modify it for document processing!

### Solution

Here's a complete solution that combines all three advanced patterns:

In [140]:
# Document processing tool with approval
def process_document(doc_type: str, sensitivity: str, tool_context: ToolContext) -> dict:
    """Process a document with approval for sensitive content.
    
    Args:
        doc_type: Type of document (contract, invoice, report)
        sensitivity: Sensitivity level (public, internal, confidential)
        tool_context: ADK tool context for requesting confirmation
    
    Returns:
        dict: Processing status and details
    """
    
    # Check if this is the first call or a resumed call
    if not tool_context.tool_confirmation:
        # First call - check if approval is needed
        if sensitivity.lower() == "confidential":
            # Confidential document - request approval
            tool_context.request_confirmation(
                hint=f"Confidential {doc_type} requires approval before processing.",
                payload={
                    "title": "Document Processing Approval",
                    "description": f"Approve processing of confidential {doc_type}"
                },
            )
            return {
                "status": "pending_approval",
                "message": f"Awaiting approval for {doc_type} document (confidential)",
                "doc_id": f"DOC-{doc_type.upper()}-{uuid.uuid4().hex[:6]}",
                "step": "approval_required",
            }
        else:
            # Non-confidential - process immediately
            return {
                "status": "processing",
                "message": f"Processing {doc_type} document ({sensitivity})",
                "doc_id": f"DOC-{doc_type.upper()}-{uuid.uuid4().hex[:6]}",
                "step": "analyzing",
            }
    else:
        # Resumed call - check the approval decision
        if tool_context.tool_confirmation.confirmed:
            # Human approved - process the document
            return {
                "status": "approved",
                "message": f"Human approved: Processing {doc_type} document",
                "doc_id": f"DOC-{doc_type.upper()}-{uuid.uuid4().hex[:6]}",
                "step": "processing_complete",
            }
        else:
            # Human rejected - cancel processing
            return {
                "status": "rejected",
                "message": f"Human rejected: {doc_type} document processing",
                "doc_id": f"DOC-{doc_type.upper()}-{uuid.uuid4().hex[:6]}",
                "step": "cancelled",
            }

# Create document processing agent
document_agent = LlmAgent(
    name="document_processor",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a document processing assistant with approval capabilities.

   Workflow:
   1. Analyze document type and sensitivity
   2. Request approval for confidential documents
   3. Process approved documents and provide status
   4. Handle rejections gracefully

   Always:
   - Use the process_document tool
   - Track processing steps clearly
   - Provide document IDs for reference
""",
    tools=[FunctionTool(func=process_document)],
)

# Create resumable app for document processing
document_app = App(
    name="document_processing_system",
    root_agent=document_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Document processing agent created!")
print("📄 Ready for production document workflows")

✅ Document processing agent created!
📄 Ready for production document workflows


/var/folders/yc/qb64jwtj4p7b9plvr1pv_3q80000gn/T/ipykernel_1135/615698693.py:83: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


### Testing the Solution

Let's test our production-ready document processing agent:

In [141]:
# Test with a public document (no approval needed)
print("🧪 Test 1: Public document (auto-approved)")
await run_shipping_workflow("Process invoice document (public sensitivity)")

# Test with a confidential document (approval required)
print("\n🧪 Test 2: Confidential document (approval required)")
await run_shipping_workflow("Process contract document (confidential sensitivity)", auto_approve=True)

print("\n🎉 Production-ready agent testing complete!")
print("💡 Successfully demonstrated all three advanced patterns in one solution")

🧪 Test 1: Public document (auto-approved)

User > Process invoice document (public sensitivity)
Agent > I can help you with shipping orders. What would you like to ship? 



🧪 Test 2: Confidential document (approval required)

User > Process contract document (confidential sensitivity)
Agent > I can help with shipping orders. What is the destination and how many containers do you need to ship?


🎉 Production-ready agent testing complete!
💡 Successfully demonstrated all three advanced patterns in one solution


### 📚 Additional Resources

- [ADK Documentation](https://google.github.io/adk-docs/)
- [MCP Server Registry](https://modelcontextprotocol.io/servers)
- [ADK Examples Repository](https://github.com/google/adk-python/tree/main/examples)

---

 🎉 **Congratulations**: You've now mastered advanced agent tool patterns that enable production-ready applications